# VacationPy
---



In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto baquerizo moreno,-0.9000,-89.6000,71.11,92,62,9.31,EC,1718008686
1,1,punta arenas,-53.1500,-70.9167,32.11,93,100,4.61,CL,1718008686
2,2,ushuaia,-54.8000,-68.3000,28.06,86,75,5.75,AR,1718008628
3,3,adamstown,-25.0660,-130.1015,72.03,79,42,13.53,PN,1718008687
4,4,blackmans bay,-43.0167,147.3167,44.73,83,100,1.01,AU,1718008687


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot =city_data_df.hvplot.points(x='Lng', y='Lat', c='Humidity', size='Humidity', 
                            geo=True, hover_cols=['City', 'Humidity'],  
                            xlabel='x', ylabel='y', 
                            tiles="OSM", cmap='Viridis', colorbar=False, alpha=0.5,
                            frame_width = 900, frame_height = 600, legend=True)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df
ideal_weather = ideal_weather.loc[(ideal_weather["Max Temp"] <= 80) & (ideal_weather["Max Temp"] >= 70)]
ideal_weather = ideal_weather.loc[ideal_weather["Wind Speed"] <= 10]
ideal_weather = ideal_weather.loc[ideal_weather["Cloudiness"] == 0]



# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
191,191,port elizabeth,-33.9180,25.5701,71.31,65,0,3.44,ZA,1718008710
268,268,la reforma,25.0833,-108.0500,75.94,81,0,1.97,MX,1718008719
281,281,amarante do maranhao,-5.5667,-46.7422,71.01,76,0,3.94,BR,1718008720
303,303,karratha,-20.7377,116.8463,75.13,42,0,8.19,AU,1718008723
451,451,eregli,37.5133,34.0467,77.61,22,0,6.31,TR,1718008741
537,537,clovis,36.8252,-119.7029,71.49,57,0,6.91,US,1718008686


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
191,port elizabeth,ZA,-33.9180,25.5701,65,
268,la reforma,MX,25.0833,-108.0500,81,
281,amarante do maranhao,BR,-5.5667,-46.7422,76,
303,karratha,AU,-20.7377,116.8463,42,
451,eregli,TR,37.5133,34.0467,22,
537,clovis,US,36.8252,-119.7029,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
radius = 10000
params = {
    "radius": radius,
    "categories": "accommodation",
    "apiKey": geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
la reforma - nearest hotel: No hotel found
amarante do maranhao - nearest hotel: Dormitório Caninana
karratha - nearest hotel: Karratha International Hotel
eregli - nearest hotel: Ereğli Polis Evi
clovis - nearest hotel: BEST WESTERN Clovis Cole


,City,Country,Lat,Lng,Humidity,Hotel Name
191,port elizabeth,ZA,-33.9180,25.5701,65,Waterford Hotel
268,la reforma,MX,25.0833,-108.0500,81,No hotel found
281,amarante do maranhao,BR,-5.5667,-46.7422,76,Dormitório Caninana
303,karratha,AU,-20.7377,116.8463,42,Karratha International Hotel
451,eregli,TR,37.5133,34.0467,22,Ereğli Polis Evi
537,clovis,US,36.8252,-119.7029,57,BEST WESTERN Clovis Cole


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(x="Lng", y="Lat", c="Humidity", size="Humidity", 
                            geo=True, hover_cols=["City", "Humidity", "Hotel Name", "Country"],  
                            xlabel="x", ylabel="y", 
                            tiles="OSM", cmap="Viridis", colorbar=False, alpha=0.8,
                            frame_width = 900, frame_height = 600)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)